# Chapter 4: Classification
predicting qualitative responses

Linear regression is not suited because any encoding implies an ordering of the categories and selecting a distance among them, which in most cases is not sensible. In the binary case, one could transform the response variable into 0s and 1s and build a linear regression model: this would lead to an OK model, even if some predictions could be larger than 1, and so it would be difficult to interpret the result of it as a probability of classifying the input as belonging to class 1.

## Logistic Regression

Logistic regression models the probability that $Y$ belongs to a particular category, e.g. $\mathbb{P}(\text{default=yes}|\text{balance}) = p(\text{balance})$ will range between 0 and 1, so, given any value for balance, a prediction for default can be made.

Logistic function: $p(X) = \dfrac{e^{\beta_0 + \beta_1X}}{1+e^{\beta_0 + \beta_1X}}$, with outputs between 0 and 1. From the definition it follows that:

$$
\dfrac{p(X)}{1-p(X)} = e^{\beta_0 + \beta_1X}.
$$
The quantity on the left is called `odds`, and ranges from 0 to $\infty$.
Taking the logarithm,
$$
\log\left({\dfrac{p(X)}{1-p(X)}}\right) = \beta_0 + \beta_1X.
$$
The quantity on the left is called `logit` or `log-odds`, which is linear in $X$. Here, increasing $X$ by 1 unit changes the log-odds by $\beta_1$, which does not correspond to the change in $p(X)$: that will depend on the current value of $X$, too.

### Estimating the coefficients